In [4]:
!python -m pip install requests tqdm

  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.6 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 751.9 kB/s eta 0:00:00
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import requests

year = 2023
day = 22

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open("cookie.txt", "r") as file:
    session = file.read()

headers = {"cookie": f"session={session}"}

with open(f"inputs/day_{str(day).zfill(2)}_input.txt", "wb") as file:
    file.write(requests.get(url, headers=headers).content)
    
aoc = ''.join(open(f"inputs/day_{str(day).zfill(2)}_input.txt", "r"))

def use_input(input_string, debug=False):
    source = [x.strip() for x in input_string.strip().split("\n")]
    if debug:
        print(source)
    G = [[x for x in line] for line in source]
    R = len(G)
    C = len(G[0])
    if debug:
        print(G[:2][:2], R,C)
    return source, G, R, C
    
lines, G, R, C = use_input(aoc, True)

['4,3,355~4,3,356', '6,7,83~6,9,83', '5,0,118~5,2,118', '3,3,327~6,3,327', '5,3,126~5,6,126', '0,7,91~2,7,91', '6,6,24~6,8,24', '6,9,195~8,9,195', '5,3,348~7,3,348', '1,2,227~1,4,227', '1,5,144~1,5,147', '9,3,76~9,5,76', '1,6,115~1,8,115', '3,5,59~3,8,59', '2,0,143~3,0,143', '7,2,170~9,2,170', '5,2,232~5,4,232', '0,1,227~1,1,227', '4,9,27~5,9,27', '7,4,42~7,6,42', '1,0,55~1,3,55', '3,1,360~3,3,360', '7,5,249~9,5,249', '0,9,345~2,9,345', '2,2,153~4,2,153', '8,1,298~9,1,298', '8,1,109~9,1,109', '7,1,33~9,1,33', '3,3,102~4,3,102', '3,1,57~3,3,57', '2,3,189~4,3,189', '0,7,96~2,7,96', '2,0,189~2,1,189', '2,0,73~2,3,73', '8,7,176~8,8,176', '8,1,113~8,1,115', '7,9,154~9,9,154', '6,5,344~6,8,344', '1,4,246~1,5,246', '7,2,5~7,4,5', '4,3,173~6,3,173', '0,7,272~0,9,272', '0,1,157~0,4,157', '0,1,49~2,1,49', '2,1,262~2,2,262', '0,1,118~3,1,118', '2,3,221~5,3,221', '6,9,81~8,9,81', '5,1,69~7,1,69', '6,4,152~6,7,152', '3,5,64~3,5,66', '6,0,166~9,0,166', '3,8,186~6,8,186', '2,7,55~5,7,55', '0,4,237~0,

In [14]:
from tqdm import tqdm

test = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9
"""

# test = """
# 0,0,3~0,0,3
# 0,0,2~0,0,2
# """

lines, G, R, C = use_input(aoc, False)

class Brick:
    def __init__(self, xs, xe, ys, ye, zs, ze):
        self.xs = xs
        self.xe = xe
        self.ys = ys
        self.ye = ye
        self.zs = zs
        self.ze = ze
        
        self.current_x = xs
        self.current_y = ys
        self.current_z = zs
        
    def size(self):
        return (self.xe - self.xs + 1) * (self.ye - self.ys  + 1 ) * (self.ze - self.zs + 1)
    
    def __iter__(self):
        return self
    
    def fall(self):
        self.ze -= 1
        self.zs -= 1
        
    def __str__(self):
        output = ""
        for x in range(self.xs, self.xe + 1):
            for y in range(self.ys, self.ye + 1):
                for z in range(self.zs, self.ze + 1):
                    output += f"({x}, {y}, {z}), "

        return output[:-2]
    
    def iterate_units(self):
        for x in range(self.xs, self.xe + 1):
            for y in range(self.ys, self.ye + 1):
                for z in range(self.zs, self.ze + 1):
                    yield (x, y, z)


bricks = []

for brick in lines:
    start, end = brick.split("~")
    
    xs, ys, zs = [int(a) for a in start.split(",")]
    xe, ye, ze = [int(b) for b in end.split(",")]
    

    obrick = Brick(xs, xe, ys, ye, zs, ze)
    #print(xs, xe, ys, ye, zs, ze, obrick.size())
    bricks.append(obrick)
    #print(obrick)

bricks.sort(key=lambda x: max(x.zs, x.ze), reverse=True)
print([str(b) for b in bricks])

def calculate_falls(ignore_index = None):
    bricks_to_fall = []
    
    for a, m_brick in enumerate(bricks):
        if ignore_index is not None:
            if a == ignore_index:
                continue
        
        can_fall = True
        
        if m_brick.zs == 1:
            can_fall = False
            continue
        
        for b, s_brick in enumerate(bricks):
            if not can_fall:
                continue
            if b < a:
                continue
            if ignore_index is not None:
                if b == ignore_index:
                    continue
            if m_brick == s_brick:
                continue
            for m_square in m_brick.iterate_units():
                fallen_square = (m_square[0], m_square[1], m_square[2] - 1)
                for s_square in s_brick.iterate_units():
                    if fallen_square == s_square:
                        can_fall = False
                        continue
                if not can_fall:
                    continue
        
        if can_fall:
            bricks_to_fall.append(m_brick)
        else:
            pass
        
    return bricks_to_fall

# for brick in bricks:
#     print(brick)
    
bricks_to_fall = None
i = 0

#while i == 0 or bricks_to_fall:

for i in tqdm(range(0, bricks[0].ze)):
    
    # if i % 10 ==0:
    #     print("fall step", i)
    bricks_to_fall = calculate_falls()
    # print(bricks_to_fall)
        
    for brick in bricks_to_fall:
        brick.fall()
            
    # for brick in bricks:
    #     print(brick)
    #i += 1
    

safe_to_disintegrate = 0
for ignore_index in tqdm(range(0, len(bricks))):
    falls = calculate_falls(ignore_index)
    if len(falls) == 0:
        safe_to_disintegrate += 1 
        
print(safe_to_disintegrate)

['(6, 0, 372), (6, 0, 373), (6, 0, 374), (6, 0, 375)', '(6, 0, 367), (6, 0, 368), (6, 0, 369)', '(5, 0, 367), (5, 0, 368), (5, 0, 369)', '(4, 0, 367), (4, 1, 367)', '(5, 0, 365), (6, 0, 365), (7, 0, 365), (8, 0, 365)', '(1, 5, 364), (2, 5, 364), (3, 5, 364)', '(2, 0, 364), (3, 0, 364), (4, 0, 364), (5, 0, 364)', '(2, 0, 363), (3, 0, 363), (4, 0, 363)', '(3, 6, 363), (4, 6, 363), (5, 6, 363)', '(2, 4, 362), (2, 5, 362), (2, 6, 362), (2, 7, 362)', '(4, 4, 362), (4, 5, 362), (4, 6, 362)', '(3, 1, 362), (3, 2, 362), (3, 3, 362)', '(3, 1, 361), (3, 2, 361), (3, 3, 361)', '(3, 1, 360), (3, 2, 360), (3, 3, 360)', '(2, 0, 360), (3, 0, 360), (4, 0, 360)', '(3, 6, 359), (3, 6, 360)', '(2, 5, 359), (3, 5, 359), (4, 5, 359)', '(1, 9, 359), (2, 9, 359), (3, 9, 359), (4, 9, 359)', '(7, 3, 359), (7, 4, 359), (7, 5, 359)', '(4, 6, 358), (4, 7, 358), (4, 8, 358), (4, 9, 358)', '(9, 3, 358), (9, 4, 358), (9, 5, 358)', '(6, 5, 358), (6, 6, 358), (6, 7, 358)', '(1, 6, 354), (1, 6, 355), (1, 6, 356), (1, 6

  3%|▎         | 10/375 [00:48<29:30,  4.85s/it]


KeyboardInterrupt: 

In [22]:
from tqdm import tqdm

test = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9
"""

# test = """
# 0,0,3~0,0,3
# 0,0,2~0,0,2
# """

lines, G, R, C = use_input(test, False)

class Brick:
    def __init__(self, xs, xe, ys, ye, zs, ze):
        self.xs = xs
        self.xe = xe
        self.ys = ys
        self.ye = ye
        self.zs = zs
        self.ze = ze
        
        self.current_x = xs
        self.current_y = ys
        self.current_z = zs
        
    def size(self):
        return (self.xe - self.xs + 1) * (self.ye - self.ys  + 1 ) * (self.ze - self.zs + 1)
    
    def __iter__(self):
        return self
    
    def fall(self):
        self.ze -= 1
        self.zs -= 1
        
    def __str__(self):
        output = ""
        for x in range(self.xs, self.xe + 1):
            for y in range(self.ys, self.ye + 1):
                for z in range(self.zs, self.ze + 1):
                    output += f"({x}, {y}, {z}), "

        return output[:-2]
    
    # def iterate_units(self):
    #     for x in range(self.xs, self.xe + 1):
    #         for y in range(self.ys, self.ye + 1):
    #             for z in range(self.zs, self.ze + 1):
    #                 yield (x, y, z)

    def overlaps(self, other_brick):
        # Check if there is an overlap in each dimension
        x_overlap = (self.xs <= other_brick.xe) and (self.xe >= other_brick.xs)
        y_overlap = (self.ys <= other_brick.ye) and (self.ye >= other_brick.ys)
        z_overlap = (self.zs <= other_brick.ze) and (self.ze >= other_brick.zs)

        # Return True if there is an overlap in all three dimensions
        return x_overlap and y_overlap and z_overlap

bricks = []

for brick in lines:
    start, end = brick.split("~")
    
    xs, ys, zs = [int(a) for a in start.split(",")]
    xe, ye, ze = [int(b) for b in end.split(",")]
    

    obrick = Brick(xs, xe, ys, ye, zs, ze)
    #print(xs, xe, ys, ye, zs, ze, obrick.size())
    bricks.append(obrick)
    #print(obrick)

bricks.sort(key=lambda x: max(x.zs, x.ze), reverse=True)
print([str(b) for b in bricks])

def calculate_falls(ignore_index = None):
    bricks_to_fall = []
    
    for a, m_brick in enumerate(bricks):
        if ignore_index is not None:
            if a == ignore_index:
                continue
        
        can_fall = True
        
        if m_brick.zs == 1:
            can_fall = False
            continue
        
        for b, s_brick in enumerate(bricks):
            if not can_fall:
                continue
            if b < a:
                continue
            if ignore_index is not None:
                if b == ignore_index:
                    continue
            if m_brick == s_brick:
                continue

            fallen_brick = Brick(m_brick.xs,m_brick.xe,m_brick.ys,m_brick.ye,m_brick.zs-1,m_brick.ze -1)

            if fallen_brick.overlaps(s_brick):
                can_fall = False

            # for m_square in m_brick.iterate_units():
            #     fallen_square = (m_square[0], m_square[1], m_square[2] - 1)
            #     for s_square in s_brick.iterate_units():
            #         if fallen_square == s_square:
            #             can_fall = False
            #             continue
            #     if not can_fall:
            #         continue
        
        if can_fall:
            bricks_to_fall.append(m_brick)
        else:
            pass
        
    return bricks_to_fall

# for brick in bricks:
#     print(brick)
    
bricks_to_fall = None
i = 0

#while i == 0 or bricks_to_fall:

for i in tqdm(range(0, bricks[0].ze)):
    
    # if i % 10 ==0:
    #     print("fall step", i)
    bricks_to_fall = calculate_falls()
    # print(bricks_to_fall)
        
    for brick in bricks_to_fall:
        brick.fall()
            
    # for brick in bricks:
    #     print(brick)
    #i += 1
    

safe_to_disintegrate = 0
for ignore_index in tqdm(range(0, len(bricks))):
    falls = calculate_falls(ignore_index)
    if len(falls) == 0:
        safe_to_disintegrate += 1 
        
print(safe_to_disintegrate)


['(1, 1, 8), (1, 1, 9)', '(0, 1, 6), (1, 1, 6), (2, 1, 6)', '(2, 0, 5), (2, 1, 5), (2, 2, 5)', '(0, 0, 4), (0, 1, 4), (0, 2, 4)', '(0, 2, 3), (1, 2, 3), (2, 2, 3)', '(0, 0, 2), (1, 0, 2), (2, 0, 2)', '(1, 0, 1), (1, 1, 1), (1, 2, 1)']


100%|██████████| 7/7 [00:00<00:00, 6957.38it/s]

5


In [24]:
safe_to_disintegrate = 0
for ignore_index in tqdm(range(0, len(bricks))):
    falls = calculate_falls(ignore_index)
    print(falls)
    safe_to_disintegrate += len(falls)
        
print(safe_to_disintegrate)

# TODO: This won't work because I need to work out the chain reaction of how many fall,
# not just the ones that fall immediately.

100%|██████████| 7/7 [00:00<00:00, 13954.43it/s]

[]
[]
[]
[]
[]
[<__main__.Brick object at 0x000002273F679EB0>, <__main__.Brick object at 0x000002273F57FC20>]
3


In [21]:
pause_bricks = bricks.copy